In [ ]:

import concurrent.futures
import itertools
print("Hello World")


### Splitting image to tiles and store into different folders

In [ ]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Ignore all FutureWarning warnings that might flood the console log
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

import os
import pandas as pd

# from encryption import encode, decode

import time
from utils import create_patches_v3, create_binary_mask, create_patches_v2, create_patches_v1, create_patches_coordinates, create_patches_coordinates_v1
from scipy.stats.contingency import crosstab
from sklearn.metrics import mutual_info_score, normalized_mutual_info_score, adjusted_mutual_info_score


# By default, the number of sub datasets is set as 3
# N_folders = 3

# Loading directory
wsi_dir = "/root/code2024/dataset/4WSI/data/"
patch_size = 224
cpu_core = 12

# Saving directory
save_dir = wsi_dir

downsize = patch_size
patch_extraction_size = patch_size
mask_overlap = 80.0
batch_size = 64
cpu_workers = cpu_core
use_prob_threshold = 0.8 # None  # whether to give final prediction {0,1} based on certain probability

# torch.manual_seed(250)
print('--------------Start--------------')
# read the files
wsi_files = os.listdir(wsi_dir)
# get all files except temp directory containing patches
wsi_files = [f for f in wsi_files if f.endswith("svs") or f.endswith("ndpi") or f.endswith("mrxs")]

print(f"Total files in {wsi_dir} directory are {len(wsi_files)}")

path = os.path.join(wsi_dir, "cnn_ensemble_updated")
if not os.path.exists(path):
    os.mkdir(path)

lst_patching_time = []

stat_NMI_graph = [] 
stat_NMI_base = []
coords = []
new_file_paths = []
# start patching process over a certain WSI
for i, f in enumerate(wsi_files):

    st_binarymask = time.time()
    new_file_path = os.path.join(path, f.split(".")[0])
    print(new_file_path)


    # just take the name not extension
    if not os.path.exists(new_file_path):
        os.mkdir(new_file_path)
        
    # find binary mask to locate tissue on WSI            
    w, h = create_binary_mask(wsi_dir, f, new_file_path, downsize=patch_size)

    # print(w/224, h/224)
    et_binarymask = time.time()

    binarymask_processtime = et_binarymask - st_binarymask
    print(f"Time spent on create_binary_mask is {binarymask_processtime} for {f}")
        
    st_patching = time.time()
    patch_folder = os.path.join(new_file_path, "patches")
    
    if not os.path.exists(patch_folder):
    # # else:
        os.mkdir(patch_folder) 
    if os.path.exists(patch_folder):
    #     os.rmdir(patch_folder)
    # elif not os.path.exists(patch_folder):
    # # else:
    #     os.mkdir(patch_folder) 
        # coordinates, id_to_coordinates = create_patches_coordinates(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #单进程
        # coordinates, id_to_coordinates, patches = create_patches_v1(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #单进程
        
        # create_patches_v3(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        
        # 1. create coordinates with multi-processing
        res = create_patches_coordinates_v1(wsi_dir, f, new_file_path, workers=cpu_workers, patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        
        # patches = create_patches_v2(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
            # patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        # print(len(coordinates))
        # print(len(patches))
        # print(id_to_coordinates)

        # print("Coordinates Done!", len(res), res[:3])
        # coordinates = list(res.values())
        coordinates = res.values()
        ids = res.keys()
        # print("Coordinates Done!", len(res), len(coordinates), coordinates[:3])
        coords.append(coordinates)
        new_file_paths.append(new_file_path)
        

# decrypting coordinates

In [ ]:
coordinates = res.values()
# ids = res.keys()
print(len(coords), len(new_file_paths))


In [ ]:
graph_split_file = f'{new_file_path}/graph_split1/'
os.mkdir(graph_split_file)

# Graph-based Splitting

In [ ]:
# 2. create sub data sets with splitting algorithm
from graph_based_splitting import graph_coloring_based_splitting, graph_coloring_based_splitting_visual, graph_coloring_based_splitting_visual1

# If ``strategy`` is a string, it must be one of the following,
# each of which represents one of the built-in strategy functions.

# * ``'largest_first'``
# * ``'random_sequential'``
# * ``'smallest_last'``
# * ``'independent_set'``
# * ``'connected_sequential_bfs'``
# * ``'connected_sequential_dfs'``
# * ``'connected_sequential'`` (alias for the previous strategy)
# * ``'saturation_largest_first'``
# * ``'DSATUR'`` (alias for the previous strategy)
# graph_distribution = []
# graph_split_file = os.mkdir(os.path.join(new_file_path, 'graph_split/'))

for coordinates, new_file_path in zip(coords, new_file_paths): 
    graph_split_file = f'{new_file_path}/graph_split1/'
    if not os.path.exists(graph_split_file):
        os.mkdir(graph_split_file)
    
    strategies = ['saturation_largest_first', 'largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
            'connected_sequential']
    
    # strategies = ['independent_set', 'connected_sequential']
    for my_strategy in strategies: 
        # d, dataset_number = graph_coloring_based_splitting1(coordinates, patch_size, strategy=my_strategy)     # graph
        d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)

        # Specify the directory path
        indexes = [item[0] for item in d.items()]
        patch_folder = f'{new_file_path}/patches/'
        patch_names = [os.path.join(patch_folder, patch) for patch in os.listdir(patch_folder)]

        # Create a DataFrame from the list of file names
        df = pd.DataFrame({"Index": indexes, "PatchName": patch_names})

        # Specify the CSV file path where the DataFrame will be saved
        csv_file_path = f'{new_file_path}/patch_names_{my_strategy}.csv'

        # Save the DataFrame to a CSV file
        df.to_csv(csv_file_path, index=False)
        print(f'File names have been saved to {csv_file_path}')

        OrgPatchName = df['PatchName']

        # store the split image names into different .csv files
        list = []
        for j in range(dataset_number):
            
            indexes = []
            names = []
            for item in d.items(): 
                # print(item) 
                if item[1] == j:
                    index = item[0]
                    # print(i, index, OrgPatchName[index])
                    indexes.append(index)
                    names.append(OrgPatchName[index])
            
            dataframe = pd.DataFrame({"Index": indexes, "PatchName": names})
            destfp = os.path.join(graph_split_file, f'graph_patchesnames_'+str(j)+my_strategy+'.csv')
            print(destfp)
            # dataframe.to_csv('patchesnames_'+str(i)+'.csv', index=False, sep=',')
            dataframe.to_csv(destfp, index=False, sep=',')

    print("---------------Well Done----------------")    

### Evenly splitting

In [ ]:
import os
print(new_file_paths)

In [ ]:
from baseline_splitting import shuffle_evenly_division

# Evenly partitionly #
# * ``is_shuffle =  True``
# * ``is_shuffle =  False``
flag = True
dataset_number = 4
for coordinates, new_file_path in zip(coords, new_file_paths): 
    baseline_split_file = f'{new_file_path}/even_split/'
    if not os.path.exists(baseline_split_file):
        os.mkdir(baseline_split_file)
    
    # os.mkdir(graph_split_file)
    # res_path ='/root/code2024/dataset/2WSI/data/cnn_ensemble_updated/TCGA-A7-A6VX-01Z-00-DX2/even_split/'
    # patch_folder = '/root/code2024/dataset/2WSI/data/cnn_ensemble_updated/TCGA-A7-A6VX-01Z-00-DX2/patches/'
    patch_folder = f'{new_file_path}/patches/'
    patch_names = [os.path.join(patch_folder, patch) for patch in os.listdir(patch_folder)]

    # print(res_path)
    print(coordinates)
    # print(res.keys())
    
    # custom_keys = [key.replace(',', '_')+'.png' for key in res.keys()]

    
    is_shuffled = f'is_shuffled_{flag}'
    base_distribution = shuffle_evenly_division(patch_names, dataset_number, is_shuffle = flag)

    base_coords = shuffle_evenly_division(coordinates, dataset_number, is_shuffle = flag)
    print(base_coords)
    # for i in base_coords:
        # print(i)
    # indexes = [item[0] for item in d.items()]
    # patch_names = [os.path.join(patch_folder, patch) for patch in os.listdir(patch_folder)]

    list = []
    for j, item in enumerate(base_distribution): 
        indexes = []
        names = []
        print(item) 
        # for item in df['PatchName']:

        indexes = [i for i in range(len(item))]
        patch_names = [os.path.join(patch_folder, patch) for patch in item]

        dataframe = pd.DataFrame({"Indexes": indexes, "PatchName": patch_names})
        
        destfp = os.path.join(baseline_split_file, f'even_patchesnames_'+str(j)+f'_{is_shuffled}.csv')
        print(destfp)
        # dataframe.to_csv('patchesnames_'+str(i)+'.csv', index=False, sep=',')
        dataframe.to_csv(destfp, index=False, sep=',')

print("---------------Well Done----------------")  


# Calculate the hist 


In [ ]:
import cv2
from matplotlib import pyplot as plt

# 加载图像
# image = cv2.imread(f'/root/code2024/dataset/2WSI/data/cnn_ensemble_updated/TCGA-A7-A6VX-01Z-00-DX2/patches/TCGA-A7-A6VX-01Z-00-DX2_28560_6720.png')
# image = cv2.imread(f'/root/code2024/dataset/3WSI/data/cnn_ensemble_updated/TCGA-A7-A6VX-01Z-00-DX2/patches/TCGA-A7-A6VX-01Z-00-DX2_28560_6720.png')
image_path = '/root/code2024/dataset/tWSI/data/cnn_ensemble_updated/TCGA-AC-A2FB-01Z-00-DX1/patches/TCGA-AC-A2FB-01Z-00-DX1_336_7728.png' 
image = cv2.imread(image_path)
# 分离颜色通道
channels = cv2.split(image)

# 颜色通道标签
colors = ('b', 'g', 'r')

# 创建颜色直方图
plt.figure()
plt.title('Color Histogram')
plt.xlabel('Bin')
plt.ylabel('# of Pixels')

# 遍历每个颜色通道
for (channel, color) in zip(channels, colors):
    # 计算该通道的直方图
    hist = cv2.calcHist([channel], [0], None, [256], [0, 256])
    
    # 绘制直方图
    plt.plot(hist, color=color)
    plt.xlim([0, 256])

# 显示直方图
# plt.savefig('./hist2.png')
plt.show()


# Combinations

# Graph Splitting